This notebook runs a `Qallse` model in simulation mode and shows how to simply visualize the results using utilities defined in module `hepqpr.qallse.plotting`.

# Import and setup

In [1]:
# import modules
import pandas as pd
import numpy as np

from hepqpr.qallse.plotting import *
from hepqpr.qallse import *
from hepqpr.qallse.dsmaker import create_dataset

# initialise the plotting module in "notebook" mode
set_notebook_mode()

# initialise the logging module
import logging
logging.basicConfig()
fmt = logging.Formatter("%(asctime)s.%(msecs)03d %(levelname)s %(name)s: %(message)s", datefmt='%H:%M:%S')
for handler in logging.getLogger().handlers: handler.setFormatter(fmt)
logging.getLogger('hepqpr').setLevel(logging.DEBUG)

0
[(-4, 4.4), (4, inf), (-inf, -4)]
0
[(0.0, 0.8)]
1
[(0.0, 0.8), (0.5, 1.3)]
2
[(0.0, 0.8), (0.5, 1.3), (1.0, 1.8)]
3
[(0.0, 0.8), (0.5, 1.3), (1.0, 1.8), (1.5, 2.3)]


# Define run config

In [2]:
# == DATASET CONFIG

dsmaker_config = dict(
    density=.02, # 1% 
)

# == INPUT CONFIG

# whether or not to add missing doublets to the input
add_missing = True 

# == RUN CONFIG

model_class = Qallse # model class to use
extra_config = dict() # configuration arguments overriding the defaults

# Generate the data

In [3]:
import tempfile

tempdir = tempfile.TemporaryDirectory()
print(f'using {tempdir.name}')
metas, path = create_dataset(output_path=tempdir.name, gen_doublets=True, random_seed=240834366, **dsmaker_config)

using /var/folders/7r/y3xzyywn4mg2tfhg0sgjqx_w0000gn/T/tmps99eqy0v


13:57:35.044 DEBUG hepqpr.qallse.dsmaker.dsmaker: Loaded 120939 hits from /Users/parkerreid/hepqpr-qallse/src/hepqpr/qallse/dsmaker/data/event000001000.
13:57:35.068 DEBUG hepqpr.qallse.dsmaker.dsmaker: Filtered hits from barrel. Remaining hits: 65518.
13:57:35.138 DEBUG hepqpr.qallse.dsmaker.dsmaker: Dropped double hits. Remaining hits: 55446.
13:57:35.171 DEBUG hepqpr.qallse.dsmaker.dsmaker: High Pt hits: 32302/1152
13:57:35.236 DEBUG hepqpr.qallse.dsmaker.dsmaker:   num_hits=1152
13:57:35.236 DEBUG hepqpr.qallse.dsmaker.dsmaker:   num_tracks=163
13:57:35.237 DEBUG hepqpr.qallse.dsmaker.dsmaker:   num_important_tracks=17
13:57:35.238 DEBUG hepqpr.qallse.dsmaker.dsmaker:   num_noise=260
13:57:35.238 DEBUG hepqpr.qallse.dsmaker.dsmaker:   random_seed=240834366
13:57:35.239 DEBUG hepqpr.qallse.dsmaker.dsmaker:   time=2020-01-03T13:57:35.236053
13:57:35.714 INFO hepqpr.qallse.dsmaker.dsmaker: Doublets (len=5626) generated in f/var/folders/7r/y3xzyywn4mg2tfhg0sgjqx_w0000gn/T/tmps99eqy0v/e

In [4]:
with open(path + '-meta.json') as f:
    print(f.read())

{
    "num_hits": 1152,
    "num_tracks": 163,
    "num_important_tracks": 17,
    "num_noise": 260,
    "random_seed": 240834366,
    "time": "2020-01-03T13:57:35.236053",
    "params": {
        "input_path": "/Users/parkerreid/hepqpr-qallse/src/hepqpr/qallse/dsmaker/data/event000001000",
        "output_path": "/var/folders/7r/y3xzyywn4mg2tfhg0sgjqx_w0000gn/T/tmps99eqy0v",
        "density": 0.02,
        "high_pt_cut": 1.0,
        "double_hits_ok": false,
        "gen_doublets": true,
        "prefix": null,
        "random_seed": 240834366,
        "phi_bounds": null,
        "min_hits_per_track": 5
    }
}


# Execute the model

## Load the data

In [5]:
# load data
dw = DataWrapper.from_path(path)
doublets = pd.read_csv(path + '-doublets.csv')
if add_missing:
    doublets = dw.add_missing_doublets(doublets)
else:
    p, r, ms = dw.compute_score(doublets)
    print(f'got {len(doublets)}.')
    print(f'  Input precision (%): {p*100:.4f}, recall (%): {r*100:.4f}, missing: {len(ms)}')

got 5626 doublets.
  Input precision (%): 2.2939, recall (%): 98.3333
    New precision (%): 2.3319


## Build the model

In [6]:
%%time

# instantiate qallse
model = model_class(dw, **extra_config)
# build the qubo
model.build_model(doublets=doublets)
sliceContainer = model.to_qubo()

13:57:35.919 DEBUG hepqpr.qallse.qallse: using config:
13:57:35.920 DEBUG hepqpr.qallse.qallse:     cheat: False
13:57:35.920 DEBUG hepqpr.qallse.qallse:     max_layer_span: 2
13:57:35.921 DEBUG hepqpr.qallse.qallse:     num_multiplier: -1
13:57:35.922 DEBUG hepqpr.qallse.qallse:     qplet_max_dcurv: 0.0001
13:57:35.922 DEBUG hepqpr.qallse.qallse:     qplet_max_strength: -0.2
13:57:35.924 DEBUG hepqpr.qallse.qallse:     qubo_bias_weight: 0
13:57:35.924 DEBUG hepqpr.qallse.qallse:     qubo_conflict_strength: 1
13:57:35.925 DEBUG hepqpr.qallse.qallse:     rz_power: 1
13:57:35.926 DEBUG hepqpr.qallse.qallse:     strength_bounds: None
13:57:35.926 DEBUG hepqpr.qallse.qallse:     tplet_max_curv: 0.0008
13:57:35.927 DEBUG hepqpr.qallse.qallse:     tplet_max_drz: 0.1
13:57:35.927 DEBUG hepqpr.qallse.qallse:     volayer_power: 2
13:57:35.928 DEBUG hepqpr.qallse.qallse:     xy_power: 1
13:57:35.928 DEBUG hepqpr.qallse.qallse:     xy_relative_strength: 0.5


[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[3]


13:57:36.194 INFO hepqpr.qallse.qallse: created 3110 doublets.
13:57:36.378 INFO hepqpr.qallse.qallse: created 563 triplets.
13:57:36.389 INFO hepqpr.qallse.qallse: created 307 quadruplets.
13:57:36.390 INFO hepqpr.qallse.qallse: Model built in 0.28s. doublets: 3110/282, triplets: 563/315, quadruplets: 307
13:57:36.395 INFO hepqpr.qallse.qallse: Dropped 2 real structures during preprocessing
13:57:36.398 DEBUG hepqpr.qallse.qallse: 
    type                      hid   reason      arg1  arg2
0  dblet              76129_96270  volayer  4.000000   7.0
1  qplet  23346_31387_38927_45412    dcurv  0.000116   NaN
13:57:36.401 INFO hepqpr.qallse.qallse: Dropped type:reason:count => dblet:volayer:1 qplet:dcurv:1 
13:57:36.410 INFO hepqpr.qallse.qallse: Qubo generated in 0.01s. Size: 271. Vars: 65, excl. couplers: 148, incl. couplers: 58
13:57:36.421 INFO hepqpr.qallse.qallse: Qubo generated in 0.02s. Size: 678. Vars: 143, excl. couplers: 391, incl. couplers: 144
13:57:36.432 INFO hepqpr.qallse.

CPU times: user 511 ms, sys: 18.5 ms, total: 529 ms
Wall time: 524 ms


## Sample the QUBO

Here, we use [_qbsolv_](https://github.com/dwavesystems/qbsolv) with default arguments. This means only classical algorithms.

In [7]:
%%time
# execute the qubo
responseContainer = model.sample_qubo_slices(Q=sliceContainer)
#response = model.sample_qubo_slices(Q=qslice)

0   0
{('37443_75645_82876', '37443_75645_82876'): 0, ('37443_44211_82876', '37443_44211_82876'): 0, ('37443_44211_75645', '37443_44211_75645'): 0, ('37454_44225_76129', '37454_44225_76129'): 0, ('38560_45453_84225', '38560_45453_84225'): 0, ('38560_45453_76929', '38560_45453_76929'): 0, ('38560_76929_84225', '38560_76929_84225'): 0, ('40321_72075_78784', '40321_72075_78784'): 0, ('18530_27240_33923', '18530_27240_33923'): 0, ('18453_26707_33903', '18453_26707_33903'): 0, ('82876_89650_96031', '82876_89650_96031'): 0, ('18530_27240_40321', '18530_27240_40321'): 0, ('18530_33923_40321', '18530_33923_40321'): 0, ('89650_96031_112054', '89650_96031_112054'): 0, ('88929_94824_111007', '88929_94824_111007'): 0, ('26678_33553_71566', '26678_33553_71566'): 0, ('26707_33903_71593', '26707_33903_71593'): 0, ('26650_33539_71357', '26650_33539_71357'): 0, ('27240_33923_72075', '27240_33923_72075'): 0, ('94824_111007_116335', '94824_111007_116335'): 0, ('27240_40321_72075', '27240_40321_72075'): 0

13:57:36.666 INFO hepqpr.qallse.qallse: QUBO of size 271 sampled in 0.22s (seed 1536043309).


0   1
{('37020_43917_82632', '37020_43917_82632'): 0, ('36971_43305_82192', '36971_43305_82192'): 0, ('36971_43305_75302', '36971_43305_75302'): 0, ('36961_75302_82192', '36961_75302_82192'): 0, ('36971_75302_82192', '36971_75302_82192'): 0, ('37018_43366_75344', '37018_43366_75344'): 0, ('37020_43917_75359', '37020_43917_75359'): 0, ('37018_75344_82231', '37018_75344_82231'): 0, ('37018_43366_82231', '37018_43366_82231'): 0, ('37020_75359_82632', '37020_75359_82632'): 0, ('37692_44398_76305', '37692_44398_76305'): 0, ('37692_76305_83410', '37692_76305_83410'): 0, ('37692_44398_83410', '37692_44398_83410'): 0, ('40765_79334_85701', '40765_79334_85701'): 0, ('40765_72807_85701', '40765_72807_85701'): 0, ('41176_72845_85723', '41176_72845_85723'): 0, ('40085_72140_78627', '40085_72140_78627'): 0, ('40321_72075_78784', '40321_72075_78784'): 0, ('41176_72845_79384', '41176_72845_79384'): 0, ('40765_72807_79334', '40765_72807_79334'): 0, ('80778_87530_93370', '80778_87530_93370'): 0, ('1874

13:57:37.053 INFO hepqpr.qallse.qallse: QUBO of size 678 sampled in 0.60s (seed 1536043309).


0   2
{('37020_43917_82632', '37020_43917_82632'): 0, ('37018_43366_75344', '37018_43366_75344'): 0, ('37020_43917_75359', '37020_43917_75359'): 0, ('37018_75344_82231', '37018_75344_82231'): 0, ('37018_43366_82231', '37018_43366_82231'): 0, ('37020_75359_82632', '37020_75359_82632'): 0, ('37263_44070_75502', '37263_44070_75502'): 0, ('37263_44070_82745', '37263_44070_82745'): 0, ('37263_75502_82745', '37263_75502_82745'): 0, ('37276_43563_75534', '37276_43563_75534'): 0, ('37276_75534_82384', '37276_75534_82384'): 0, ('37276_43563_82384', '37276_43563_82384'): 0, ('37948_44583_76015', '37948_44583_76015'): 0, ('37871_44524_76430', '37871_44524_76430'): 0, ('37948_76015_83183', '37948_76015_83183'): 0, ('37948_44583_83183', '37948_44583_83183'): 0, ('41176_72845_85723', '41176_72845_85723'): 0, ('40085_72140_78627', '40085_72140_78627'): 0, ('41176_72845_79384', '41176_72845_79384'): 0, ('80100_86293_92029', '80100_86293_92029'): 0, ('23248_37276_43563', '23248_37276_43563'): 0, ('8077

13:57:37.459 INFO hepqpr.qallse.qallse: QUBO of size 760 sampled in 1.01s (seed 1536043309).


0   3
{('37263_44070_75502', '37263_44070_75502'): 0, ('37263_44070_82745', '37263_44070_82745'): 0, ('37263_75502_82745', '37263_75502_82745'): 0, ('37276_43563_75534', '37276_43563_75534'): 0, ('37276_75534_82384', '37276_75534_82384'): 0, ('37276_43563_82384', '37276_43563_82384'): 0, ('37948_44583_76015', '37948_44583_76015'): 0, ('37948_76015_83183', '37948_76015_83183'): 0, ('37948_44583_83183', '37948_44583_83183'): 0, ('38927_45412_84457', '38927_45412_84457'): 0, ('38945_77205_84220', '38945_77205_84220'): 0, ('38927_77184_84457', '38927_77184_84457'): 0, ('38927_45412_77184', '38927_45412_77184'): 0, ('38945_45440_77205', '38945_45440_77205'): 0, ('38945_45440_84220', '38945_45440_84220'): 0, ('80100_86293_92029', '80100_86293_92029'): 0, ('23248_37276_43563', '23248_37276_43563'): 0, ('77205_84220_91243', '77205_84220_91243'): 0, ('23248_30508_37276', '23248_30508_37276'): 0, ('23259_30515_37948', '23259_30515_37948'): 0, ('23248_30508_43563', '23248_30508_43563'): 0, ('8133

13:57:37.867 INFO hepqpr.qallse.qallse: QUBO of size 510 sampled in 1.42s (seed 1536043309).


CPU times: user 1.41 s, sys: 9.21 ms, total: 1.42 s
Wall time: 1.42 s


## Parse the results

In [8]:
# get all output doublets
all_doublets = model.process_sample_slices(responseContainer=responseContainer)
# recreate tracks and resolve remaining conflicts
final_tracks, final_doublets = TrackRecreaterD().process_results(all_doublets)

13:57:37.879 INFO hepqpr.qallse.track_recreater: Found 0 conflicting doublets


# Evaluate the results

## Print statistics and scores

In [9]:
# stats about the qbsolv run
#en0 = dw.compute_energy(Q)
#en = response.record.energy[0]

#print(f'SAMPLE -- energy: {en:.4f}, ideal: {en0:.4f} (diff: {en-en0:.6f})')
#occs = response.record.num_occurrences
#print(f'          best sample occurrence: {occs[0]}/{occs.sum()}')

# scores
p, r, missings = dw.compute_score(final_doublets)
print(f'SCORE  -- precision (%): {p * 100}, recall (%): {r * 100}, missing: {len(missings)}')
trackml_score = dw.compute_trackml_score(final_tracks)
print(f'          tracks found: {len(final_tracks)}, trackml score (%): {trackml_score * 100}')

SCORE  -- precision (%): 100.0, recall (%): 92.5, missing: 9
          tracks found: 22, trackml score (%): 90.3625248517814


### Plot the results

You have the following at your disposal:
```python
iplot_results(dw, final_doublets, missings, dims=list('xy'))
iplot_results_tracks(dw, final_doublets, missings, dims=list('zr'))
iplot_results_any(dw, dw.real_doublets, dims=list('xy'))
```

For 3D plots, simply define `dims` as 3-dimensional. For example:
```python
iplot_results(dw, final_doublets, missings, dims=list('zxy'))
```

In [10]:
iplot_results(dw, final_doublets, missings, dims=list('xy'))

In [11]:
iplot_results(dw, final_doublets, missings, dims=list('zr'))

In [12]:
iplot_results(dw, final_doublets, missings, dims=list('zxy'))